In [1]:
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
import seaborn as sns
import pandas as pd
import requests
import os
import re
import time
import numpy as np
import holidays
from datetime import datetime, timedelta
from scipy.stats import skew, kurtosis
from scipy.optimize import brentq
import scipy.stats as si
from scipy.stats import norm
import matplotlib.patches as patches
import ast
from polygon import RESTClient
from polygon.rest.models.request import RequestOptionBuilder
import warnings
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
from requests.exceptions import HTTPError
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
from datetime import datetime
import matplotlib.patches as patches
API_KEY = "HsOX1rl8t3B1AJfl4O2uqscUp372l7EF"
us_holidays = holidays.US(years=2024)
dst_start = pd.to_datetime('2024-03-10')
dst_end = pd.to_datetime('2024-11-03')
fila =6 # Determina el ancho de la caja en la que buscaremos el máx y el mín.
# no confundir fila que determina la caja del IB con row que determina el núm. de índice cuando la caja es rota.

In [2]:
def get_time_based_on_dst(fecha):
    if dst_start <= fecha <= dst_end:
        return pd.to_datetime('13:30:00').time()
    else:
        return pd.to_datetime('14:30:00').time()

def next_working_day(date, DTE):
    date_obj = datetime.strptime(date, '%Y-%m-%d')
    next_day = date_obj + timedelta(days= DTE)
    while next_day.weekday() in (5, 6) or next_day in us_holidays:
        next_day += timedelta(days=1)
    return next_day.strftime('%Y-%m-%d')

def get_stock_price(date, expiration_day):
    url = f"https://api.polygon.io/v2/aggs/ticker/SPY/range/1/minute/{date}/{expiration_day}?adjusted=true&sort=asc&apiKey={API_KEY}"
    #print(url)  # For debugging purposes
    #print(f"Trading Date:    {date}")
    #print(f"Expiration Date: {expiration_day}")
    response = requests.get(url)

    if response.status_code == 200:
        try:
            data = response.json()
            if 'results' in data:
                return [(result['t'], result['c']) for result in data['results']]
            else:
                raise ValueError("Response JSON does not contain 'results' key")
        except ValueError as e:
            print(f"Error parsing JSON: {e}")
    else:
        print(f"Request failed with status code {response.status_code}")
    return []

def get_stock_data_for_date(date, DTE):
    expiration_day = next_working_day_1(date, DTE)
    option_data = get_stock_price(date, expiration_day)
    df = pd.DataFrame(option_data, columns=['timestamp', 'stock_price'])
    df['fecha'] = pd.to_datetime(df['timestamp'], unit='ms')
    df['time_filter'] = df['fecha'].apply(get_time_based_on_dst)
    df = df[df['fecha'].dt.time >= df['time_filter']]
    df['stock_price'] = round(df['stock_price'],2)
    df = df.reset_index(drop=True)
    df.drop(columns=['time_filter'], inplace=True)
    return df

In [3]:
def get_option_price(ticker, date, expiration_day, option_type):
    url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/minute/{date}/{expiration_day}?adjusted=true&sort=asc&apiKey={API_KEY}"
    print(url)
    response = requests.get(url)
    if response.status_code == 200:
        try:
            data = response.json()
            # Ensure 'results' key is present in the JSON response
            if 'results' in data:
                results = data['results']
                df_option = pd.DataFrame(results)
                
                # Check what columns are in df_option
                print(df_option.columns)
                
                # Rename columns if they exist
                if all(col in df_option.columns for col in ['o', 'c', 'l', 'h']):
                    df_option = df_option.rename(columns={
                        't': 'timestamp',
                        'o': 'open',
                        'c': 'close',
                        'l': 'low',
                        'h': 'high'
                    })
                    df_option['option_code'] = ticker
                    df_option['option_type'] = option_type                
                    return df_option[['timestamp', 'option_code', 'option_type', 'close']]
                else:
                    print("Expected columns are not present in the returned data.")
        except ValueError as e:
            print(f"Error parsing JSON: {e}")
    else:
        print(f"Request failed with status code {response.status_code}")
    return None

In [4]:
def plot_option_chart(df_option, strike_B, option_type_B, option_ticker_B, fila, option_type_A, strike_A):
    ovenight_index = int(len(df_option) / 2) - 40
    end_index = len(df_option) - 2

    df_option['fecha'] = df_option['fecha'].astype(str)
    fig, (ax1, ax3) = plt.subplots(nrows=2, ncols=1, figsize=(12, 9), gridspec_kw={'height_ratios': [3, 1], 'hspace': 0.04})
    ax2 = ax1.twinx()
    
    ax1.plot(df_option['fecha'], df_option['close_B'], color='green', label='B', linewidth=2.0)
    ax1.plot(df_option['fecha'], df_option['close_A'], color='olive', label='A', linewidth=1.4)
    
    ax2.plot(df_option['fecha'], df_option['stock_price'], color='blue', alpha=1.0, linewidth=0.8)
    ax1.set_ylabel('Prima', color='green')
    ax2.set_ylabel('Stock Price', color='blue')
    ax1.tick_params(axis='y', labelcolor='green')
    ax2.tick_params(axis='y', labelcolor='blue')
    ax1.yaxis.grid(True, alpha=0.6)
    ax1.xaxis.grid(False)
    
    ax1.tick_params(axis='x', which='both', length=0, labelbottom=False)
    
    ax1.axvline(x=fila, color='grey', alpha=0.56, linestyle='-', linewidth=0.7)
    ax1.axvline(x=ovenight_index, color='grey', alpha=0.5, linestyle='-', linewidth=0.7)
    
    ax2.axhline(y=strike_B, color='green', alpha=0.7, linestyle='--')
    ax2.axhline(y=strike_A, color='olive', alpha=0.7, linestyle='--')

    
    ax1.plot(df_option['fecha'][fila], df_option['close_B'][fila], 'ko', markersize=9)
    
    ax1.plot(df_option['fecha'][ovenight_index], df_option['close_B'][ovenight_index], 'ro', markersize=9)
    ax1.plot(df_option['fecha'][end_index], df_option['close_B'][end_index], 'ro', markersize=9)
    
    ax1.set_title(f'Main: {option_ticker_B} - A:{option_type_A}{strike_A}  B:{option_type_B}{strike_B}')
    
    ax3.fill_between(df_option['fecha'], df_option['cum_return'], where=(df_option['cum_return'] >= 0),
                     color='green', alpha=0.5, label=' ')
    ax3.fill_between(df_option['fecha'], df_option['cum_return'], where=(df_option['cum_return'] < 0),
                     color='red', alpha=0.5, label=' ')
    ax3.set_ylabel('Beneficio Acumulado')
    ax3.tick_params(axis='y')
    ax3.xaxis.set_major_locator(plt.MaxNLocator(10))
    
    for tick in ax3.get_xticklabels():
        tick.set_rotation(45)
        tick.set_horizontalalignment('right')

    ax3.yaxis.grid(True, alpha=0.6)
    ax3.xaxis.grid(False)
    ax1.legend(loc='upper center')

    output_dir = 'chart_oputput_sim'
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'sell_the_swan_{timestamp}.png'
    output_path = os.path.join(output_dir, filename)
    plt.savefig(output_path)
    plt.show()

In [5]:
def generate_option_ticker(expiration_day_B, option_type_B, strike_B):
    # Convert expiration_day_B to the desired format (YYMMDD without '20' and '-')
    expiration_day_B_formatted = expiration_day_B.strftime('%y%m%d')
    strike_B_formatted = f'{strike_B:05d}'
    option_ticker_B = f'O:SPY{expiration_day_B_formatted}{option_type_B}{strike_B_formatted}000'
    return option_ticker_B

DISEÑO

In [6]:
date = '2024-08-06'

# OPTION A
option_type_A = 'P'
distance_A =    -3
DTE_A =          1

# OPTION B
option_type_B = 'P'
distance_B =    -1
DTE_B =          1

# OPTION C
option_type_C = 'C'
distance_C =     1
DTE_C =          1


In [7]:
# OPTION A - elegir vencimiento
df = get_stock_data_for_date(date, DTE_A)
expiration_day_A = next_working_day(date, DTE_A)   # INDICARE AQUÍ LA CALENDAR PARA LA COBERTURA
expiration_day_A = pd.to_datetime(expiration_day_A).date()
current_stock_price = int(df.iloc[fila]['stock_price'])
strike_A = current_stock_price + distance_A
option_ticker_A = generate_option_ticker(expiration_day_A, option_type_A, strike_A)
print (option_ticker_A)
df_option_A = get_option_price(option_ticker_A, date, expiration_day_A, option_type_A)
df_option_A.rename(columns={'option_code': 'option_code_A', 'close': 'close_A', 'option_type': 'option_type_A'}, inplace=True)

NameError: name 'next_working_day_1' is not defined

In [ ]:
# OPTION B - elegir vencimiento
df = get_stock_data_for_date(date, DTE_B)
expiration_day_B = next_working_day(date, DTE_B)   # INDICARE AQUÍ LA CALENDAR PARA LA COBERTURA
expiration_day_B = pd.to_datetime(expiration_day_B).date()
current_stock_price = int(df.iloc[fila]['stock_price'])
strike_B = current_stock_price + distance_B
option_ticker_B = generate_option_ticker(expiration_day_B, option_type_B, strike_B)
print (option_ticker_B)
df_option_B = get_option_price(option_ticker_B, date, expiration_day_B, option_type_B)
df_option_B.rename(columns={'option_code': 'option_code_B', 'close': 'close_B', 'option_type': 'option_type_B'}, inplace=True)

In [ ]:
# OPTION C - elegir vencimiento
df = get_stock_data_for_date(date, DTE_C)
expiration_day_C = next_working_day(date, DTE_C)   # INDICARE AQUÍ LA CALENDAR PARA LA COBERTURA
expiration_day_C = pd.to_datetime(expiration_day_C).date()
current_stock_price = int(df.iloc[fila]['stock_price'])
strike_C = current_stock_price + distance_C
option_ticker_C = generate_option_ticker(expiration_day_C, option_type_C, strike_C)
print (option_ticker_B)
df_option_C = get_option_price(option_ticker_C, date, expiration_day_C, option_type_C)
df_option_C.rename(columns={'option_code': 'option_code_C', 'close': 'close_C', 'option_type': 'option_type_C'}, inplace=True)

In [ ]:
print('\nOPCION A')
print(f'Fecha entrada      {date}')
print(f'Fecha vencimiento  {expiration_day_A}')
print(f'Precio Subyacente         {current_stock_price}')
print(f'Precio Strike {option_type_A}{distance_A}         {strike_A}')
print(f'DTE                         {DTE_A}\n')
print('OPCION B')
print(f'Fecha entrada      {date}')
print(f'Fecha vencimiento  {expiration_day_B}')
print(f'Precio Subyacente         {current_stock_price}')
print(f'Precio Strike {option_type_B}{distance_B}         {strike_B}')
print(f'DTE                         {DTE_B}\n')
print('OPCION C')
print(f'Fecha entrada      {date}')
print(f'Fecha vencimiento  {expiration_day_C}')
print(f'Precio Subyacente         {current_stock_price}')
print(f'Precio Strike {option_type_C}{distance_C}          {strike_C}')
print(f'DTE                         {DTE_C}\n')

MERGE, PROFIT AND PLOT

In [ ]:
df_option = pd.merge(df_option_B, df, on='timestamp', how='left')
df_option = pd.merge(df_option, df_option_A,  on='timestamp', how='left')

In [ ]:
coste_B = df_option['close_B'].iloc[0]
df_option['cum_return_B'] = coste_B - df_option['close_B']

coste_A = df_option['close_A'].iloc[0]
df_option['cum_return_A'] = df_option['close_A'] - coste_A 

df_option['cum_return'] = df_option['cum_return_B'] + df_option['cum_return_A']
df_option = df_option.fillna(method='ffill')

In [ ]:
plot_option_chart(df_option, strike_B, option_type_B, option_ticker_B, fila, option_type_A, strike_A)

In [ ]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'options_sim_{timestamp}.csv'
df_option.to_csv(filename, index=False)

In [ ]:
df_option